## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_Database.csv file.
fname = os.path.join("../Weather_Database", "WeatherPy_Database.csv")
city_data_df = pd.read_csv(fname)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tadine,NC,-21.5500,167.8833,77.88,86,100,2.71,heavy intensity rain
1,1,Dikson,RU,73.5069,80.5464,20.82,98,100,21.39,light snow
2,2,Vao,NC,-22.6667,167.4833,78.67,85,100,9.33,moderate rain
3,3,Busselton,AU,-33.6500,115.3333,55.99,91,3,11.12,clear sky
4,4,Kupang,ID,-10.1667,123.5833,78.49,83,93,4.81,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tadine,NC,-21.5500,167.8833,77.88,86,100,2.71,heavy intensity rain
2,2,Vao,NC,-22.6667,167.4833,78.67,85,100,9.33,moderate rain
3,3,Busselton,AU,-33.6500,115.3333,55.99,91,3,11.12,clear sky
4,4,Kupang,ID,-10.1667,123.5833,78.49,83,93,4.81,light rain
5,5,Weligama,LK,5.9667,80.4167,75.67,82,100,4.47,light rain
6,6,Rikitea,PF,-23.1203,-134.9692,78.01,77,74,14.58,broken clouds
7,7,Bengkulu,ID,-3.8004,102.2655,76.35,84,100,4.52,heavy intensity rain
8,8,Castro,BR,-24.7911,-50.0119,78.03,77,4,4.92,clear sky
9,9,New Norfolk,AU,-42.7826,147.0587,59.00,91,70,2.98,broken clouds
10,10,Victoria,HK,22.2855,114.1577,53.01,30,97,10.00,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                415
City                   415
Country                411
Lat                    415
Lng                    415
Max Temp               415
Humidity               415
Cloudiness             415
Wind Speed             415
Current Description    415
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                411
City                   411
Country                411
Lat                    411
Lng                    411
Max Temp               411
Humidity               411
Cloudiness             411
Wind Speed             411
Current Description    411
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tadine,NC,77.88,heavy intensity rain,-21.5500,167.8833,
2,Vao,NC,78.67,moderate rain,-22.6667,167.4833,
3,Busselton,AU,55.99,clear sky,-33.6500,115.3333,
4,Kupang,ID,78.49,light rain,-10.1667,123.5833,
5,Weligama,LK,75.67,light rain,5.9667,80.4167,
6,Rikitea,PF,78.01,broken clouds,-23.1203,-134.9692,
7,Bengkulu,ID,76.35,heavy intensity rain,-3.8004,102.2655,
8,Castro,BR,78.03,clear sky,-24.7911,-50.0119,
9,New Norfolk,AU,59.00,broken clouds,-42.7826,147.0587,
10,Victoria,HK,53.01,overcast clouds,22.2855,114.1577,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... s

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", float("NaN"), inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vao,NC,78.67,moderate rain,-22.6667,167.4833,Hotel Kou-Bugny
3,Busselton,AU,55.99,clear sky,-33.6500,115.3333,Observatory Guest House
4,Kupang,ID,78.49,light rain,-10.1667,123.5833,Hotel On The Rock by Prasanthi
5,Weligama,LK,75.67,light rain,5.9667,80.4167,Weligama Bay Resort
6,Rikitea,PF,78.01,broken clouds,-23.1203,-134.9692,Pension Maro'i
7,Bengkulu,ID,76.35,heavy intensity rain,-3.8004,102.2655,Grage Hotel Bengkulu
8,Castro,BR,78.03,clear sky,-24.7911,-50.0119,CHACARA BAILLY
9,New Norfolk,AU,59.00,broken clouds,-42.7826,147.0587,The Shingles Riverside Cottages
10,Victoria,HK,53.01,overcast clouds,22.2855,114.1577,Mini Hotel Central
11,Butaritari,KI,81.52,broken clouds,3.0707,172.7902,Isles Sunset Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))